<a href="https://colab.research.google.com/github/MitjaGo/Utube/blob/main/UTUBE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- STEP 1: Install dependencies ---
!pip install yt-dlp pydub tqdm ipywidgets > /dev/null

# --- STEP 2: Import libraries ---
import os
import subprocess
from concurrent.futures import ThreadPoolExecutor
import ipywidgets as widgets
from IPython.display import display, FileLink, clear_output
import shutil

# --- STEP 3: Prepare output folder ---
output_dir = "mp3_downloads"
os.makedirs(output_dir, exist_ok=True)

# --- STEP 4: Global list to store songs ---
search_terms = []

# --- STEP 5: Define the download function ---
def download_song(term, progress_widgets, song_index):
    safe_name = term.replace(" ", "_")[:60]
    expected_file = os.path.join(output_dir, f"{safe_name}.mp3")

    # Update current song label
    progress_widgets['current_song'].value = f"⏳ Downloading ({song_index+1}/{progress_widgets['total']}): {term}"

    if os.path.exists(expected_file):
        progress_widgets['skipped'].value += 1
        return (term, "skipped")

    cmd = [
        "yt-dlp",
        f"ytsearch1:{term}",
        "-x",
        "--audio-format", "mp3",
        "--audio-quality", "0",
        "-o", f"{output_dir}/%(title)s.%(ext)s"
    ]

    try:
        subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        progress_widgets['success'].value += 1
        return (term, "success")
    except subprocess.CalledProcessError:
        progress_widgets['failed'].value += 1
        return (term, "failed")

# --- STEP 6: Create UI widgets ---
file_upload = widgets.FileUpload(
    accept='.txt',
    multiple=False,
    description="Upload Song List",
    style={'button_color': '#1f77b4', 'font_weight': 'bold'}
)

download_button = widgets.Button(
    description="Start Download",
    button_style="success",
    icon="download",
    layout=widgets.Layout(width='200px', margin='10px')
)

progress_success = widgets.IntProgress(
    value=0, min=0, max=1, description='✔ Success', bar_style='success', orientation='horizontal', layout=widgets.Layout(width='300px')
)
progress_skipped = widgets.IntProgress(
    value=0, min=0, max=1, description='⏭ Skipped', bar_style='info', orientation='horizontal', layout=widgets.Layout(width='300px')
)
progress_failed = widgets.IntProgress(
    value=0, min=0, max=1, description='❌ Failed', bar_style='danger', orientation='horizontal', layout=widgets.Layout(width='300px')
)
current_song_label = widgets.Label(value="Status: Waiting for upload", layout=widgets.Layout(margin='10px 0px'))
file_status_label = widgets.Label(value="", layout=widgets.Layout(margin='10px 0px'))

output_area = widgets.Output(layout=widgets.Layout(border='1px solid lightgray', padding='10px', width='650px', height='300px', overflow='auto'))

# --- STEP 7: Layout cards ---
control_card = widgets.VBox([file_upload, file_status_label, download_button], layout=widgets.Layout(border='1px solid #ccc', padding='15px', margin='10px', width='350px'))
progress_card = widgets.VBox([progress_success, progress_skipped, progress_failed, current_song_label], layout=widgets.Layout(border='1px solid #ccc', padding='15px', margin='10px', width='350px'))

dashboard = widgets.HBox([control_card, progress_card])
full_layout = widgets.VBox([dashboard, output_area])

# --- STEP 8: Observe file upload with multiple encoding support ---
download_button.disabled = True  # disable until file uploaded

def read_uploaded_file(content_bytes):
    """Try multiple encodings to decode file content."""
    for enc in ['utf-8', 'latin1', 'cp1250', 'iso-8859-2']:
        try:
            return content_bytes.decode(enc)
        except UnicodeDecodeError:
            continue
    # If all fail, fallback to latin1 ignoring errors
    return content_bytes.decode('latin1', errors='ignore')

def on_file_upload_change(change):
    global search_terms
    if file_upload.value:
        uploaded_filename = list(file_upload.value.keys())[0]
        content_bytes = file_upload.value[uploaded_filename]['content']
        content_str = read_uploaded_file(content_bytes)
        search_terms = [line.strip() for line in content_str.splitlines() if line.strip()]

        if search_terms:
            file_status_label.value = f"✅ File uploaded: {uploaded_filename} ({len(search_terms)} songs)"
            current_song_label.value = f"🎶 {len(search_terms)} songs ready to download!"
            download_button.disabled = False
        else:
            file_status_label.value = f"❌ No valid songs found in {uploaded_filename}"
            current_song_label.value = "Status: Waiting for valid upload"
            download_button.disabled = True

file_upload.observe(on_file_upload_change, names='value')

# --- STEP 9: Define download process ---
def start_download(b):
    if not search_terms:
        with output_area:
            clear_output(wait=True)
            print("❌ No songs loaded. Please upload a valid .txt file first.")
        return

    total_songs = len(search_terms)
    progress_success.max = total_songs
    progress_skipped.max = total_songs
    progress_failed.max = total_songs

    progress_widgets = {
        'success': progress_success,
        'skipped': progress_skipped,
        'failed': progress_failed,
        'current_song': current_song_label,
        'total': total_songs
    }

    with output_area:
        clear_output(wait=True)
        print(f"🚀 Starting download of {total_songs} songs...\n")

    # Multi-threaded downloads
    results = []
    MAX_WORKERS = 4
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = [executor.submit(download_song, term, progress_widgets, idx) for idx, term in enumerate(search_terms)]
        for future in futures:
            results.append(future.result())

    # Summary
    success = [term for term, status in results if status == "success"]
    skipped = [term for term, status in results if status == "skipped"]
    failed = [term for term, status in results if status == "failed"]

    with output_area:
        print("\n✅ Downloading complete!\n")
        print(f"📊 Summary:")
        print(f"  ✔️ Successful: {len(success)}")
        print(f"  ⏭️ Skipped: {len(skipped)}")
        print(f"  ❌ Failed: {len(failed)}")
        if failed:
            print("\n⚠️ Could not download:")
            for f in failed:
                print(f"   - {f}")

        # Zip results
        shutil.make_archive("mp3_downloads", 'zip', output_dir)
        print("\n📦 Download your MP3s:")
        display(FileLink("mp3_downloads.zip"))

# --- STEP 10: Wire button ---
download_button.on_click(start_download)

# --- STEP 11: Display full dashboard ---
display(full_layout)
